In [2]:
!nvidia-smi

Wed Apr 27 05:29:40 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!python --version

Python 3.7.13


In [ ]:
!cd /content/drive/MyDrive/ast2vec_tf

In [ ]:
# cd /content/drive/MyDrive/major-proj/ast2vec_tf

[Errno 2] No such file or directory: '/content/drive/MyDrive/major-proj/ast2vec_tf'
/content


In [3]:
import os
os.chdir('/content/drive/MyDrive/ast2vec_tf')
# import tensorflow as tf
# import tensorflow.keras.layers as layers
# from collections import OrderedDict

# import python_ast_utils
# import tree as tree
# import tree_grammar
# import numpy as np



# KERAS

In [ ]:


class RTGAELSTM(tf.keras.Model):
  def __init__(self, grammar, dim):
        super(RTGAELSTM, self).__init__()
        self.grammar = grammar
        self.dim = dim
        # set up a tree parser for the given grammar. We need this for
        # training the autoencoder
        self.parser_ = tree_grammar.TreeParser(grammar)


        # set up a map from symbols to indices
        self.symbol_to_idx_ = {}
        symbols = list(self.grammar._alphabet.keys())
        for i in range(len(symbols)):
            self.symbol_to_idx_[symbols[i]] = i

        # set up a GRU to encode siblings
        self.enc_siblings_ = layers.LSTM(self.dim,return_state=True)
        # set up a GRU to encode to parents
        self.enc_parents_  = layers.LSTM(self.dim,return_state=True)
        # set up a GRU to decode siblings
        self.dec_siblings_ = layers.LSTM(self.dim,return_state=True)
        self.dec_siblings_out_ = layers.Dense(self.dim, input_shape=(self.dim,))
        # set up a GRU to decode from parents
        # self.dec_parents_ = 
        self.dec_parents_  = layers.LSTM(self.dim,return_state=True)
        # set up linear layers to decide which grammar rule to take
        # from a given nonterminal symbol
        self.dec_cls_ = OrderedDict()
        for nont in self.grammar._nonts:
            self.dec_cls_[nont] = layers.Dense(len(self.grammar._rules[nont]), input_shape=(self.dim,), activation = 'softmax')
        # also set up classifiers for optional and starred rules
        for lhs in self.grammar._rules:
            for rhs in self.grammar._rules[lhs]:
                for nont in rhs[1]:
                    if nont not in self.dec_cls_ and isinstance(nont, str) and (nont.endswith('*') or nont.endswith('?')):
                        self.dec_cls_[nont] = layers.Dense(2, input_shape=(self.dim,))

  def encode(self, x):
        # first encode the sequence of children recursively
        if len(x._children) == 0:
            h_child = tf.Variable(tf.zeros([1,self.dim]))
        else:
            H_child = tf.Variable(tf.zeros([len(x._children), self.dim]))
            for k in range(len(x._children)):
                # aa=aa[2].assign(1)
                H_child = H_child[k,:].assign(self.encode(x._children[k])[0])
                # H_child[k, :] = self.encode(x._children[k])
            _, h_child,_ = self.enc_siblings_(tf.expand_dims(H_child, axis=0))
            # h_child = tf.expand_dims(h_child, axis=[0])
        # then, encode to a parent representation
        indices = [self.symbol_to_idx_[x._label]]
        depth = len(self.symbol_to_idx_)
        label_encoding = tf.expand_dims(tf.one_hot(indices, depth), axis=1)
        _, h_parent,_ = self.enc_parents_(inputs=label_encoding, initial_state=[h_child,h_child])
        return h_parent

  def decode(self, h, max_size = None):
        x, N = self.decode_(h, self.grammar._start, max_size)
        return x

  def decode_(self, h, nont, max_size = None):
        
        # first, decide the grammar rule that shall be applied
        r = np.argmax(self.dec_cls_[nont](h))
        rule = self.grammar._rules[nont][r]
        # generate a new tree object with the label determined by the
        # grammar rule
        label = rule[0]
        child_nonts = rule[1]
        x = tree.Tree(label)
        # map from parent to child state

        indices = [self.symbol_to_idx_[x._label]]
        depth = len(self.symbol_to_idx_)
        label_encoding = tf.expand_dims(tf.one_hot(indices, depth), axis=1)
        _, h_child,_ = self.dec_parents_(inputs=label_encoding, initial_state=[h,h])
        # generate all children that are determined by the grammar rule
        child_nonts = rule[1]
        N = 1
        if max_size is not None:
            max_size -= 1
        for k in range(len(child_nonts)):
            if isinstance(child_nonts[k], str) and child_nonts[k].endswith('*'):
                # if the next child is starred, continue decoding children for
                # this nonterminal until the classifier says otherwise
                while max_size is None or max_size > 0:
                    continue_flag = np.argmax(self.dec_cls_[child_nonts[k]](h_child))
                    if continue_flag == 0:
                        break
                    # infer state for the next child
                    hk = self.dec_siblings_out_(h_child)
                    # decode it recursively and append it
                    y, Nk = self.decode_(hk, child_nonts[k][:-1], max_size)
                    x._children.append(y)
                    # adjust the remaining maximum size
                    if max_size is not None:
                        max_size -= Nk
                    N += Nk
                    # update the state
                    hk = tf.expand_dims(hk,axis=0)
                    _, h_child,_ = self.dec_siblings_(inputs=hk, initial_state=[h_child,h_child])
                # once all children for the starred nonterminal are decoded,
                # continue with the next child nonterminal
                continue
            elif isinstance(child_nonts[k], str) and child_nonts[k].endswith('?'):
                # if the next child is optional, check if it shall be decoded
                continue_flag = np.argmax(self.dec_cls_[child_nonts[k]](h_child))
                if continue_flag == 0:
                    # if not, continue right away
                    continue
                nont = child_nonts[k][:-1]
            else:
                nont = child_nonts[k]
            # if the max size is exceeded, leave the nonterminals
            if max_size is not None and max_size <= 0:
                x._children.append(tree.Tree(nont))
                continue
            # at this point, we know that the child shall be decoded regularly.
            # First, we infer the state for the next child
            hk = self.dec_siblings_out_(h_child)
            # then, we decode it recursively and append it
            y, Nk = self.decode_(hk, nont, max_size)
            x._children.append(y)
            # adjust the remaining maximum size
            if max_size is not None:
                max_size -= Nk
            N += Nk
            # update the state
            hk = tf.expand_dims(hk,axis=0)
            _, h_child, _ = self.dec_siblings_(inputs=hk, initial_state=[h_child,h_child])
        # return
        return x, N

  def decode_forced_(self, h, nont, seq, t, score_list, decoding_list = None):
        # first, decide the current scores
        if decoding_list is not None:
            decoding_list.append(h)
        score_list.append(self.dec_cls_[nont](h))
        # then, retrieve the rule that ought to be applied
        r = seq[t]
        t += 1
        rule = self.grammar._rules[nont][r]
        # generate a new tree object with the label determined by the
        # grammar rule
        label = rule[0]
        child_nonts = rule[1]
        # map from parent to child state

        indices = [self.symbol_to_idx_[label]]
        depth = len(self.symbol_to_idx_)
        label_encoding = tf.expand_dims(tf.one_hot(indices, depth), axis=1)
        # label_encoding = torch.zeros(len(self.symbol_to_idx_))
        # label_encoding[self.symbol_to_idx_[label]] = 1.
        _, h_child,_ = self.dec_parents_(inputs=label_encoding, initial_state=[h,h])
        # generate all children that are determined by the grammar rule
        child_nonts = rule[1]
        for k in range(len(child_nonts)):
            if isinstance(child_nonts[k], str) and child_nonts[k].endswith('*'):
                # if the next child is starred, continue decoding children for
                # this nonterminal until the rule sequence says otherwise
                score_list.append(self.dec_cls_[child_nonts[k]](h_child))
                while seq[t] == 1:
                    t += 1
                    # infer state for the next child
                    hk = self.dec_siblings_out_(h_child)
                    # decode the current child
                    t = self.decode_forced_(hk, child_nonts[k][:-1], seq, t, score_list, decoding_list)
                    # update the state
                    hk = tf.expand_dims(hk,axis=0)
                    _, h_child,_ = self.dec_siblings_(inputs=hk, initial_state=[h_child,h_child])
                    score_list.append(self.dec_cls_[child_nonts[k]](h_child))
                t += 1
                # once all children for the starred nonterminal are decoded,
                # continue with the next child nonterminal
                continue
            elif isinstance(child_nonts[k], str) and child_nonts[k].endswith('?'):
                # if the next child is optional, check if it shall be decoded
                score_list.append(self.dec_cls_[child_nonts[k]](h_child))
                if seq[t] == 0:
                    t += 1
                    continue
                t += 1
                nont = child_nonts[k][:-1]
            else:
                nont = child_nonts[k]
            # at this point, we know that the child shall be decoded regularly.
            # First, we infer the state for the next child
            hk = self.dec_siblings_out_(h_child)
            # then, we decode it recursively
            t = self.decode_forced_(hk, nont, seq, t, score_list, decoding_list)
            # update the state
            hk = tf.expand_dims(hk,axis=0)
            _, h_child, _ = self.dec_siblings_(inputs=hk, initial_state=[h_child,h_child])
        # return
        return t

  def pad_seq(self, t):
    t_pad = []
    maxshape = max([j.shape[1] for j in t])
    for i in t:
      t2 = tf.zeros([1,maxshape-i.shape[1]],dtype=i.dtype)
      i_pad = tf.concat([i, t2], 1)
      t_pad.append(i_pad)
    return tf.squeeze(tf.stack(t_pad, axis=0), axis = 1), maxshape

  def one_hot_seq(self,t,depth):
    t_one = []
    for i in t:
        i_one = tf.one_hot([i], depth)
        t_one.append(i_one)
    return tf.squeeze(tf.stack(t_one, axis=0), axis = 1)

  
  def compute_loss(self, x):
        h = self.encode(x)
        nont, seq = self.parser_.parse_tree(x)
        scores = []
        self.decode_forced_(h, nont, seq, 0, scores)
        scores,depth = self.pad_seq(scores)
        seq = self.one_hot_seq(seq,depth)
        loss = tf.keras.losses.categorical_crossentropy(seq,scores,from_logits=False, label_smoothing=0, axis=-1)
        tot_loss = sum(loss.numpy())
        return tot_loss

  def call(self, inputs):
    x = self.compute_loss(inputs)
    return x


In [ ]:
model = RTGAELSTM(python_ast_utils.grammar, dim = 256)

In [ ]:
def encode_trees(model, trees):
      X = np.zeros((len(trees), 256))
      # iterate over all trees
      for i in range(len(trees)):
          # encode the current tree
          x = model.encode(trees[i])
          # convert the vector to numpy and store it in the matrix
      # return the matrix
      return x


In [ ]:
programs     = ['arr = input().split()\n']
trees, programs_to_trees = python_ast_utils.parse_asts(programs, filter_uniques = True)
vec=encode_trees(model, trees)
print(trees,"\n\n\n\n",vec)

In [ ]:
trees

[Module(Assign(Name, Call(Attribute(Call(Name)))))]

In [ ]:
def generate_vec():
  rng = np.random.default_rng()
  vec = tf.convert_to_tensor(rng.random((1, 256)), dtype=tf.float32)
  # vec = ( (vec  - 0 ) / (1 - 0) ) * (1 - -1) + -1
  return vec

In [ ]:
def decode_points(model, X, max_size = 10):
    trees = model.decode(X, max_size = max_size)
    return trees



In [ ]:
vec = generate_vec()
tre=decode_points(model,vec)
tre

Module(AugAssign(Set, FloorDiv, GeneratorExp(DictComp(Constant, Constant, comprehension(SetComp(expr), expr)))))

In [ ]:
model.compute_loss(tre)

KeyError: ignored

In [ ]:
# vec=encode_trees(modellstm, trees)

In [ ]:
print("hi rahul457")

hi rahul457


In [ ]:
x=trees[0]
h = model.encode(x)
nont, seq = model.parser_.parse_tree(x)
scores = []
print(model.decode_forced_(h, nont, seq, 0, scores))

15


In [ ]:
seq

[0, 1, 4, 1, 20, 0, 13, 17, 13, 20, 0, 0, 0, 0, 0]

In [ ]:
type(trees)

list

In [ ]:
scores

[<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[1.]], dtype=float32)>,
 <tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[0.48622945, 0.5137705 ]], dtype=float32)>,
 <tf.Tensor: shape=(1, 20), dtype=float32, numpy=
 array([[0.05069299, 0.04961899, 0.04831997, 0.04962244, 0.05026175,
         0.0506347 , 0.05018949, 0.04982139, 0.05331923, 0.05073491,
         0.05121638, 0.05068263, 0.04808311, 0.05126059, 0.04890196,
         0.04874926, 0.0518473 , 0.04632362, 0.05115911, 0.04856017]],
       dtype=float32)>,
 <tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[0.5056311 , 0.49436888]], dtype=float32)>,
 <tf.Tensor: shape=(1, 23), dtype=float32, numpy=
 array([[0.04322062, 0.04179545, 0.04185909, 0.04485264, 0.04236766,
         0.04236579, 0.04630024, 0.04411554, 0.04274567, 0.04441154,
         0.0417267 , 0.0466552 , 0.04557517, 0.04108223, 0.0436444 ,
         0.04425921, 0.04496698, 0.04318183, 0.0436477 , 0.04192689,
         0.04115065, 0.04254494, 0.04560382]]

In [ ]:
seq

[0, 1, 4, 1, 20, 0, 13, 17, 13, 20, 0, 0, 0, 0, 0]

In [ ]:
14,24
14,1

(14, 1)

In [ ]:
a = pad_seq(scores)
a.shape

TensorShape([15, 24])

In [ ]:
b=one_hot_seq(seq,24)
b.shape

TensorShape([15, 24])

In [ ]:
trees

[Module(Assign(Name, Call(Attribute(Call(Name)))))]

In [ ]:
model.compute_loss(trees[0])

24.47753328084947

AttributeError: ignored

# PYTORCH

In [4]:
import torch

if torch.cuda.is_available():
  print("Selecting GPU")
  device = torch.device("cuda")
else:
  device = torch.device("cpu")

Selecting GPU


In [9]:
import tree as tree
import tree_grammar
import python_ast_utils
import numpy as np

In [6]:
class SimpleLSTM(torch.nn.Module):
    """ A easier-to-use version of the gated recurrent unit (Cho et al., 2014).

    Attributes
    ----------
    _dim_in: int
        The input dimensionality.
    _dim_hid: int
        The state dimensionality.
    _gru: class torch.nn.GRU
        The actual GRU network.

    """
    def __init__(self, dim_in, dim_hid):
        super(SimpleLSTM, self).__init__()
        self._dim_in = dim_in
        self._dim_hid = dim_hid
        self._dim_cell = dim_hid
        self._lstm = torch.nn.LSTM(self._dim_in, self._dim_hid)

    def forward(self, x, h = None):
        """ Updates the state given the current state and the current input.

        Parameters
        ----------
        x: class torch.Tensor
            The current input with dimensionality self._dim_in
        h: class torch.Tensor (default = zero vector)
            The current state with dimensionality self._dim_hid.

        Returns
        -------
        h: class torch.Tensor
            The next state with dimensionality self._dim_hid, computed as
            described above.

        """
        # fill state with zeros if not given
        if h is None:
            h = torch.zeros(self._dim_hid)
        # extend with empty dimensions
        h = h.unsqueeze(0).unsqueeze(1)
        # print(h.shape)
        # extend x with empty dimensions
        x = x.unsqueeze(0).unsqueeze(1)
        # compute the next state
        _, (h,c) = self._lstm(x, (h, h))
        # return it
        return h.squeeze(1).squeeze(0)

In [7]:
class RTGAE(torch.nn.Module):
    """ An autoencoder for trees over a given grammar.

    Parameters
    ----------
    grammar: class tree_grammar.TreeGrammar
        The grammar from which trees are generated
    dim: int
        The encoding dimensionality.

    Attributes
    ----------

    """
    def __init__(self, grammar, dim):
        super(RTGAE, self).__init__()
        self.grammar = grammar
        self.dim = dim

        # set up a tree parser for the given grammar. We need this for
        # training the autoencoder
        self.parser_ = tree_grammar.TreeParser(grammar)

        # set up a map from symbols to indices
        self.symbol_to_idx_ = {}
        symbols = list(self.grammar._alphabet.keys())
        for i in range(len(symbols)):
            self.symbol_to_idx_[symbols[i]] = i

        # set up a GRU to encode siblings
        self.enc_siblings_ = torch.nn.LSTM(self.dim, self.dim)
        # set up a GRU to encode to parents
        self.enc_parents_  = SimpleLSTM(len(symbols), self.dim)
        # set up a GRU to decode siblings
        self.dec_siblings_ = SimpleLSTM(self.dim, self.dim)
        self.dec_siblings_out_ = torch.nn.Linear(self.dim, self.dim)
        # set up a GRU to decode from parents
        self.dec_parents_  = SimpleLSTM(len(symbols), self.dim)
        # set up linear layers to decide which grammar rule to take
        # from a given nonterminal symbol
        self.dec_cls_ = torch.nn.ModuleDict()
        for nont in self.grammar._nonts:
            self.dec_cls_[nont] = torch.nn.Linear(self.dim, len(self.grammar._rules[nont]))
        # also set up classifiers for optional and starred rules
        for lhs in self.grammar._rules:
            for rhs in self.grammar._rules[lhs]:
                for nont in rhs[1]:
                    if nont not in self.dec_cls_ and isinstance(nont, str) and (nont.endswith('*') or nont.endswith('?')):
                        self.dec_cls_[nont] = torch.nn.Linear(self.dim, 2)


    def encode(self, x):
        """ Encodes the given input tree recursively by first
        encoding the child sequence to a vector and then the state
        representing the child sequence to a parent state.

        Parameters
        ----------
        x: class tree.Tree
            An input tree.

        Returns
        -------
        h: class torch.Tensor
            A self.dim dimensional vectorial encoding of the input tree.

        """
        # first encode the sequence of children recursively
        if len(x._children) == 0:
            h_child = torch.zeros(self.dim)
        else:
            H_child = torch.zeros(len(x._children), self.dim)
            for k in range(len(x._children)):
                H_child[k, :] = self.encode(x._children[k])
            _, (h_child,_) = self.enc_siblings_(H_child.unsqueeze(1))
            h_child = h_child.squeeze(1).squeeze(0)
        # then, encode to a parent representation
        label_encoding = torch.zeros(len(self.symbol_to_idx_))
        label_encoding[self.symbol_to_idx_[x._label]] = 1.
        h_parent = self.enc_parents_(label_encoding, h_child)
        return h_parent


    def decode(self, h, max_size = None):
        """ Decodes the given vector into a tree.

        Parameters
        ----------
        h: class torch.Tensor
            A self.dim dimensional vectorial encoding of some tree.
        max_size: int (default = None)
            A maximum number of decoding steps to prevent endless recursion.

        Returns
        -------
        x: class tree.Tree
            The decoded tree.

        """
        x, N = self.decode_(h, self.grammar._start, max_size)
        return x

    def decode_(self, h, nont, max_size = None):
        """ Decodes the given vector into a tree.

        Parameters
        ----------
        h: class torch.Tensor
            A self.dim dimensional vectorial encoding of some tree.
        nont: str
            The nonterminal symbol from which decoding shall start.
        max_size: int (default = None)
            A maximum number of decoding steps to prevent endless recursion.

        Returns
        -------
        x: class tree.Tree
            The decoded tree.
        N: int
            The size of the decoded tree.

        """
        # first, decide the grammar rule that shall be applied
        r = torch.argmax(self.dec_cls_[nont](h))
        rule = self.grammar._rules[nont][r]
        # generate a new tree object with the label determined by the
        # grammar rule
        label = rule[0]
        child_nonts = rule[1]
        x = tree.Tree(label)
        # map from parent to child state
        label_encoding = torch.zeros(len(self.symbol_to_idx_))
        label_encoding[self.symbol_to_idx_[x._label]] = 1.
        h_child = self.dec_parents_(label_encoding, h)
        # generate all children that are determined by the grammar rule
        child_nonts = rule[1]
        N = 1
        if max_size is not None:
            max_size -= 1
        for k in range(len(child_nonts)):
            if isinstance(child_nonts[k], str) and child_nonts[k].endswith('*'):
                # if the next child is starred, continue decoding children for
                # this nonterminal until the classifier says otherwise
                while max_size is None or max_size > 0:
                    continue_flag = torch.argmax(self.dec_cls_[child_nonts[k]](h_child))
                    if continue_flag == 0:
                        break
                    # infer state for the next child
                    hk = self.dec_siblings_out_(h_child)
                    # decode it recursively and append it
                    y, Nk = self.decode_(hk, child_nonts[k][:-1], max_size)
                    x._children.append(y)
                    # adjust the remaining maximum size
                    if max_size is not None:
                        max_size -= Nk
                    N += Nk
                    # update the state
                    h_child = self.dec_siblings_(hk, h_child)
                # once all children for the starred nonterminal are decoded,
                # continue with the next child nonterminal
                continue
            elif isinstance(child_nonts[k], str) and child_nonts[k].endswith('?'):
                # if the next child is optional, check if it shall be decoded
                continue_flag = torch.argmax(self.dec_cls_[child_nonts[k]](h_child))
                if continue_flag == 0:
                    # if not, continue right away
                    continue
                nont = child_nonts[k][:-1]
            else:
                nont = child_nonts[k]
            # if the max size is exceeded, leave the nonterminals
            if max_size is not None and max_size <= 0:
                x._children.append(tree.Tree(nont))
                continue
            # at this point, we know that the child shall be decoded regularly.
            # First, we infer the state for the next child
            hk = self.dec_siblings_out_(h_child)
            # then, we decode it recursively and append it
            y, Nk = self.decode_(hk, nont, max_size)
            x._children.append(y)
            # adjust the remaining maximum size
            if max_size is not None:
                max_size -= Nk
            N += Nk
            # update the state
            h_child = self.dec_siblings_(hk, h_child)
        # return
        return x, N

    def decode_forced_(self, h, nont, seq, t, score_list, decoding_list = None):
        """ Decodes the given vector into a tree, forced by the given sequence
        of rules and accumulates the rule scores of the model along the way.

        Parameters
        ----------
        h: class torch.Tensor
            A self.dim dimensional vectorial encoding of some tree.
        nont: str
            The nonterminal symbol from which decoding shall start.
        seq: list
            A sequence of grammar rules.
        t: int
            The current index in seq.
        score_list: list
            The output list of scores.
        decoding_list: list (default = None)
            The output list for decodings along the way.

        Returns
        -------
        t: int
            The index in seq after decoding h.

        """
        # first, decide the current scores
        if decoding_list is not None:
            decoding_list.append(h)
        score_list.append(self.dec_cls_[nont](h))
        # then, retrieve the rule that ought to be applied
        r = seq[t]
        t += 1
        rule = self.grammar._rules[nont][r]
        # generate a new tree object with the label determined by the
        # grammar rule
        label = rule[0]
        child_nonts = rule[1]
        # map from parent to child state
        label_encoding = torch.zeros(len(self.symbol_to_idx_))
        label_encoding[self.symbol_to_idx_[label]] = 1.
        h_child = self.dec_parents_(label_encoding, h)
        # generate all children that are determined by the grammar rule
        child_nonts = rule[1]
        for k in range(len(child_nonts)):
            if isinstance(child_nonts[k], str) and child_nonts[k].endswith('*'):
                # if the next child is starred, continue decoding children for
                # this nonterminal until the rule sequence says otherwise
                score_list.append(self.dec_cls_[child_nonts[k]](h_child))
                while seq[t] == 1:
                    t += 1
                    # infer state for the next child
                    hk = self.dec_siblings_out_(h_child)
                    # decode the current child
                    t = self.decode_forced_(hk, child_nonts[k][:-1], seq, t, score_list, decoding_list)
                    # update the state
                    h_child = self.dec_siblings_(hk, h_child)
                    score_list.append(self.dec_cls_[child_nonts[k]](h_child))
                t += 1
                # once all children for the starred nonterminal are decoded,
                # continue with the next child nonterminal
                continue
            elif isinstance(child_nonts[k], str) and child_nonts[k].endswith('?'):
                # if the next child is optional, check if it shall be decoded
                score_list.append(self.dec_cls_[child_nonts[k]](h_child))
                if seq[t] == 0:
                    t += 1
                    continue
                t += 1
                nont = child_nonts[k][:-1]
            else:
                nont = child_nonts[k]
            # at this point, we know that the child shall be decoded regularly.
            # First, we infer the state for the next child
            hk = self.dec_siblings_out_(h_child)
            # then, we decode it recursively
            t = self.decode_forced_(hk, nont, seq, t, score_list, decoding_list)
            # update the state
            h_child = self.dec_siblings_(hk, h_child)
        # return
        return t

    def compute_loss(self, x):
        """ Autoencodes the given tree and computes the classification loss
        between selecting the correct node labels and the choice of the current
        model.

        Parameters
        ----------
        x: class tree.Tree
            An input tree.

        Returns
        -------
        loss: class torch.Tensor
            The classification loss for choosing the correct node labels.

        """
        # encode the tree
        h = self.encode(x)
        # find the actual rule sequence that should be chosen by using a
        # tree parser
        nont, seq = self.parser_.parse_tree(x)
        # find the rules that the current autoencoder would have chosen instead
        scores = []
        self.decode_forced_(h, nont, seq, 0, scores)
        # transform the actual score list and the desired chosen rules
        # into torch tensors
        scores = torch.nn.utils.rnn.pad_sequence(scores, True)
        seq    = torch.tensor(seq, dtype=torch.long)
        # compute crossentropy loss
        loss = torch.nn.functional.cross_entropy(scores, seq, reduction = 'sum')
        # return it
        return loss


    def forward(self, x):
        encoded = self.encode(x)
        decoded = self.decode(encoded)
        return decoded
    

    # def compute_acc(self, x):
        

In [ ]:
import os

# load the data first. We create a list of all programs and a list of traces,
# where each trace is a list of program indices
programs     = ['']
traces       = []
# start loading the data
student_dirs = list(sorted(os.listdir('mock_dataset')))
for student_dir in student_dirs:
    # initialize a new trace for the student which starts at the empty program
    trace = [0]
    # load all steps of this student
    steps = list(sorted(os.listdir(f'mock_dataset/{student_dir}')))
    for step in steps:
        # load the current program
        with open(f'mock_dataset/{student_dir}/{step}') as program_file:
            trace.append(len(programs))
            programs.append(program_file.read())
    # append the trace
    traces.append(trace)
print('read %d traces with %d programs' % (len(traces), len(programs)))

In [10]:
XX = np.random.rand(1000000,256)
XX = ( (XX  - 0 ) / (1 - 0) ) * (1 - -1) + -1

In [11]:
import ast2vec
model = ast2vec.load_model('ast2vec.pt')

RuntimeError: ignored

In [10]:
data=[]
with open("dataset.txt") as file:

    for line in file:
        data.append(line.strip())
        

In [11]:
len(data)

496009

In [12]:
import python_ast_utils

trees = python_ast_utils.parse_asts(data[:1000],verbose=True)
print('After compilation, %d unique syntax trees were left' % len(trees))

starting parsing


100%|██████████| 1000/1000 [00:02<00:00, 354.23it/s]

After compilation, 1000 unique syntax trees were left


In [13]:
len(trees)
#type(trees[0])

1000

In [14]:
model = RTGAE(python_ast_utils.grammar, dim = 256)
optimizer = torch.optim.Adam(model.parameters(),
                             lr=1e-3, 
                             weight_decay=1e-5)

In [ ]:
vec= model.encode(trees[2])
print(vec)

In [16]:
num_epochs = 10
from tqdm import tqdm
# programs     = ['arr = input().split()\n', 'arr = input().split()\n', 'arr = input().split()\n']
# trees, programs_to_trees = python_ast_utils.parse_asts(programs, filter_uniques = True)
outputs = []
progbar=tqdm
for epoch in range(num_epochs):
    for x in progbar(range(len(trees[:100]))):
        loss = model.compute_loss(trees[x])
        #acc = model.compute_acc(x)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # print(loss)
        # i = 0
    
    # for x in val_trees:
    #   val_loss = model.compute_loss(x)
    #   val_acc = model.compute_acc(x)    


    print(f'Epoch:{epoch+1}, Loss:{loss.item():.4f}')#, VAL Loss:{val_loss.item():.4f}')
    outputs.append((epoch, loss))


100%|██████████| 100/100 [01:20<00:00,  1.25it/s]


Epoch:1, Loss:62.2686


100%|██████████| 100/100 [01:20<00:00,  1.24it/s]


Epoch:2, Loss:57.1091


100%|██████████| 100/100 [01:26<00:00,  1.15it/s]


...

Epoch:8, Loss:48.4968


100%|██████████| 100/100 [01:21<00:00,  1.23it/s]


Epoch:9, Loss:48.2614


100%|██████████| 100/100 [01:21<00:00,  1.22it/s]

Epoch:10, Loss:45.7326
